In [ ]:
%%capture
import os

!python -m pip install --upgrade pip
!pip install typing_extensions
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [ ]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!
    r = 16,           # 0보다 큰 어떤 숫자도 선택 가능! 8, 16, 32, 64, 128이 권장됩니다.
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",   # 바이어스를 지원합니다.
    random_state = 3407
)

In [ ]:
import json
from datasets import Dataset

In [ ]:
# JSON 파일 읽기
def load_and_preprocess_data(file_path='train.json'):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 데이터 플랫화 및 정제
    processed_data = []
    
    for entry in data:
        for review in entry['reviews']:
            # 불필요한 특수문자 및 따옴표 제거
            question = review['question'].strip().replace("'", "").replace('"', '')
            answer = review['answer'].strip().replace("'", "").replace('"', '')
            comment = review['comment'].strip().replace("'", "").replace('"', '')
            
            # 데이터 정제 및 구조화
            processed_data.append({
                'question': question,
                'answer': answer,
                'comment': comment,
                # instruction 형식으로 변환
                'instruction': f"다음 질문에 대한 답변을 작성해주세요:\n{question}",
                # input/output 형식으로 변환
                'input': question,
                'output': answer,
                'feedback': comment
            })

    # Dataset 객체로 변환
    dataset = Dataset.from_list(processed_data)
    
    # 학습용 프롬프트 템플릿 적용
    def apply_prompt_template(examples):
        prompts = []
        for i in range(len(examples['question'])):
            prompt = f"""### 질문:
{examples['question'][i]}


In [ ]:
### 모범 답변:
{examples['answer'][i]}

### 피드백:
{examples['comment'][i]}
<|endoftext|>"""
            prompts.append(prompt)
        
        return {'text': prompts}

    # 프롬프트 템플릿 적용
    formatted_dataset = dataset.map(apply_prompt_template, batched=True)
    
    return formatted_dataset

# 데이터셋 생성
dataset = load_and_preprocess_data()

# 데이터셋 정보 출력
print(f"데이터셋 크기: {len(dataset)}")
print("\n데이터셋 예시:")
print(dataset[0]['text'])


In [ ]:
dataset['train'][0]

In [ ]:
from trl import SFTTrainer, SFTConfig

tokenizer.padding_side = "right"  # 토크나이저의 패딩을 오른쪽으로 설정합니다.

# SFTTrainer를 사용하여 모델 학습 설정
trainer = SFTTrainer(
    model=model,  # 학습할 모델
    tokenizer=tokenizer,  # 토크나이저
    train_dataset=dataset['train'],  # 학습 데이터셋
    eval_dataset=dataset['train'],# 테디노트에서 추가된 eval dataset
    dataset_text_field="text",  # 데이터셋에서 텍스트 필드의 이름
    dataset_num_proc=2,  # 데이터 처리에 사용할 프로세스 수
    packing=False,  # 짧은 시퀀스에 대한 학습 속도를 5배 빠르게 할 수 있음
    args=SFTConfig(
        max_seq_length=7994,  # 최대 시퀀스 길이
        per_device_train_batch_size=2,  # 각 디바이스당 훈련 배치 크기
        gradient_accumulation_steps=4,  # 그래디언트 누적 단계
        warmup_steps=5,  # 웜업 스텝 수
        num_train_epochs=3,  # 훈련 에폭 수
        max_steps=100,  # 최대 스텝 수 # 공식문서에는 60
        logging_steps=1,  # logging 스텝 수
        learning_rate=2e-4,  # 학습률
        fp16=not torch.cuda.is_bf16_supported(),  # fp16 사용 여부, bf16이 지원되지 않는 경우에만 사용
        bf16=torch.cuda.is_bf16_supported(),  # bf16 사용 여부, bf16이 지원되는 경우에만 사용
        optim="adamw_8bit",  # 최적화 알고리즘
        weight_decay=0.01,  # 가중치 감소
        lr_scheduler_type="cosine",  # 학습률 스케줄러 유형 # 공식은 linear
        seed=123,  # 랜덤 시드 # 공식은 3407
        output_dir="outputs",  # 출력 디렉토리
    ),
)


In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
# @title Training model
trainer_stats = trainer.train()

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [ ]:
from transformers import TextStreamer

_ = model.generate(
    **tokenizer(["'차세대 예지 보전 설비기술 도입' 21살 때, 삼성전자 화성캠퍼스 신설 현장에서 일하며 반도체 설비에 관한 관심을 키우게 되었습니다. 당시, 공장 내부 가스관 설치 현장에서 배관 시공을 간접적으로 경험하며 설비 유지 보수에 관심을 키우게 되었습니다. 아무리 완벽한 반도체 이론이 있더라도 원활한 설비 공급 없이는 제품을 구현할 수 없다는 점을 느꼈습니다. 삼성전자 반도체 연구소는 세계 최고의 기술을 연구하여 글로벌 메모리 반도체 1위 자리를 30년간 지켜내고 있습니다. 이런 설비기술의 선구자인 삼성전자에서 차세대 예지 보전 설비관리 특허보유라는 꿈을 이루고 싶습니다."], return_tensors = "pt").to("cuda"),
    max_new_tokens = 512, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)